# Generative Approach for Machine Reading Comprehension (MRC)

This Jupyter notebook evaluates the performance of generative Question-Answering transformers models on Pirá Dataset. 

Generative models produce the answer with the help of the supporting text.

Check the full GitHub at: https://github.com/C4AI/Pira

## Imports

In [ ]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
import pandas as pd 
import pandas as pd
from simpletransformers.t5 import T5Model
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys



## Choose whether also to train the model or just evaluate it.

In [ ]:
DO_TRAINING = True


## Dataset information

Here we set some values necessary to load the dataset

PATH_BASE -> Dataset path


SUPPORTING_TEXT_COLUMN -> Indicates the Supporting Text Column. Use "abstract" for English or "abstract_translated_pt" for Portuguese.

QUESTION_COLUMN -> Indicates the Answer Column. Use "question_en_origin" for English or "question_pt_origin" for Portuguese.


ANSWER_COLUMN -> Indicates the Answer Column. Use "answer_en_origin" for English or "answer_pt_origin" for Portuguese.



In [1]:

PATH_BASE = './Data/test.csv'

SUPPORTING_TEXT_COLUMN = "abstract"
QUESTION_COLUMN_NAME = "question_en_origin"
ANSWER_COLUMN_NAME = "answer_en_origin"



## Loading Dataset

In [3]:

pd_dataset_train = pd.read_csv(PATH_BASE + "train.csv", index_col = 0)
pd_dataset_train["input_text"] = pd_dataset_train[QUESTION_COLUMN_NAME] + "  context: " + pd_dataset_train[SUPPORTING_TEXT_COLUMN]
pd_dataset_train["target_text"] = pd_dataset_train[ANSWER_COLUMN_NAME] 
pd_dataset_train["prefix"] = "question"
pd_dataset_train = pd_dataset_train[['prefix', 'input_text', 'target_text']]

pd_dataset_val = pd.read_csv(PATH_BASE + "validation.csv", index_col = 0)
pd_dataset_val["input_text"] = pd_dataset_val[QUESTION_COLUMN_NAME] + "  context: " + pd_dataset_val[SUPPORTING_TEXT_COLUMN]
pd_dataset_val["target_text"] = pd_dataset_val[ANSWER_COLUMN_NAME] 
pd_dataset_val["prefix"] = "question"
pd_dataset_val = pd_dataset_val[['prefix', 'input_text', 'target_text']]



pd_dataset_test = pd.read_csv(PATH_BASE + "test.csv", index_col = 0)
pd_dataset_test["input_text"] = "question: " + pd_dataset_test[QUESTION_COLUMN_NAME] + "  context: " + pd_dataset_test[SUPPORTING_TEXT_COLUMN]
pd_dataset_test["target_text"] = pd_dataset_test[ANSWER_COLUMN_NAME] 
pd_dataset_test = pd_dataset_val[['prefix', 'input_text', 'target_text']]

test_input = pd_dataset_test["input_text"].values.tolist()
test_target_2 = pd_dataset_test["target_text"].values.tolist()
test_target = []
for i in test_target_2:
    test_target.append([i])

In [4]:
pd_dataset_train.head()

prefix                                         input_text  \
id_qa                                                                
A160   question  What method can be used to estimate air–sea se...   
B2252  question  Why is mercury considered one of the ten most ...   
B2044  question  What are the biological benefits of marine hyd...   
A931   question  What is the most productive oil province in Br...   
B2045  question  What is the importance of the cultural practic...   

                                             target_text  
id_qa                                                     
A160                    The turbulent covariance method.  
B2252  Because in humans, exposure can be harmful to ...  
B2044  Marine hydrates with seeping gas are high biod...  
A931                                    The Campos basin  
B2045  They are essential to the maintenance and crea...

## Model Parameters

MODEL_CHECKPOINT -> Local or from HuggingFace model.

In [5]:
MODEL_CHECKPOINT = "t5-large"

MAX_SEQ_LENGHT = 512
MAX_OUTPUT_LENGHT = 16
EPOCHS = 40
BATCH_SIZE = 1
GRADIENT_ACCUMULATION = 32
LEARNING_RATE = 2e-5

In [6]:

model_args = {
    "preprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": MAX_SEQ_LENGHT,
    "train_batch_size": BATCH_SIZE,
    "gradient_accumulation_steps": GRADIENT_ACCUMULATION,
    "num_train_epochs": EPOCHS,
    "no_save": False,
    "save_eval_checkpoints": False,
    "learning_rate": LEARNING_RATE,
    "use_multiprocessing": True,
    "save_steps": 2000,
    "save_model_every_epoch": False,
    "evaluate_during_training": True,
    "fp16": True,
    "target_max_len": MAX_OUTPUT_LENGHT
}

## Training the model

In case DO_TRAINING was set to True

In [7]:
model = T5Model("t5", MODEL_CHECKPOINT, args=model_args)

if DO_TRAINING:
    model.train_model(pd_dataset_train, eval_data=pd_dataset_val)

  0%|          | 0/1806 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/home/blab-answerer/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5

Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

Running Epoch 0 of 40:   0%|          | 0/1806 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


...

  0%|          | 0/225 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Running Epoch 39 of 40:   0%|          | 0/1806 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


## Generating answers

In [9]:

preds = model.predict(test_input)

preds[0]

Generating outputs:   0%|          | 0/29 [00:00<?, ?it/s]

/home/blab-answerer/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/225 [00:00<?, ?it/s]

'relationship to Atlantic Ocean surface temperature anomalies'

## Evaluationg script

SQuAD evaluation script: https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py

Modified slightly for this notebook since we do not remove articles to remain consistent for both Portuguese and English

In [8]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

## Performing Evaluation

In [10]:
evaluate(test_target, preds)

{'exact_match': 10.666666666666666, 'f1': 41.2153884877768}